<a href="https://colab.research.google.com/github/anne-urai/ddm_mediation/blob/main/fit_simdata_HDDMnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# First, set Runtime -> Runtime type -> GPU for fitting

# https://hddm.readthedocs.io/en/latest/lan_tutorial.html
!pip install scikit-learn
!pip install cython
!pip install pymc

# after making changes to the hddm forked repo, factory reset runtime
!pip install -U --no-deps git+https://github.com/anne-urai/hddm
!pip install -U --no-deps git+https://github.com/hddm-devs/kabuki


# MODULE IMPORTS ----
# warning settings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Stats functionality
from statsmodels.distributions.empirical_distribution import ECDF

# to download data
from google.colab import files
import pprint

# HDDM
import hddm

     |████████████████████████████████| 385 kB 3.9 MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1361161 sha256=2420cebc862609b0c7937439b55e175d2d3b69b73b734ef339ee0cdabf404cc1
  Stored in directory: /root/.cache/pip/wheels/3b/9b/1d/2a2ddbe80519db2698745f0988ce50c36ad38aac886a4ee0ca
Successfully built pymc
  Cloning https://github.com/anne-urai/hddm to /tmp/pip-req-build-s0t4qe4g
  Running command git clone -q https://github.com/anne-urai/hddm /tmp/pip-req-build-s0t4qe4g
  Running command git submodule update --init --recursive -q
  Created wheel for HDDM: filename=HDDM-0.9.3-cp37-cp37m-linux_x86_64.whl size=3875318 sha256=a125796c93816fdb36fe1b48676603945dcb15d97b1f39ba5ca22562ec5d2bda
  Stored in directory: /tmp/pip-ephem-wheel-cache-hqxgwgkn/wheels/fa/1c/d6/4d91002b5305b3dc76ca13cd496b9bbfd0fc9350d27ab615ca
Successfully built HDDM
  Cloning https://github.com/hddm-devs/kabuki to /tmp/pip-req-build-f_7x8e7z
  Running command git clone -q https:/

In [5]:
for eff_x in ['v']: #, 'z', 'vz', 'no']:
  for eff_m in ['v']: #, 'z', 'vz', 'no']:

   # grab datafile from GitHub
    data = pd.read_csv('https://github.com/anne-urai/ddm_mediation/raw/main/data/data_df_X%s_M%s.csv'%(eff_x, eff_m))

    # Set up the regressors
    reg_model_v = {'model': 'v ~ 1 + S + M + X', 'link_func': lambda x: x}
    # reg_model_z = {'model': 'z ~ 1 + prevresp', 'link_func': lambda x: x}
    reg_descr = [reg_model_v]

    model = 'ddm' # start simple
    n_samples = 1000
    hddmnn_model = hddm.HDDMnn(data,
                               model = model,
                               include = hddm.simulators.model_config[model]['hddm_include'],
                               p_outlier = 0.05,
                               is_group_model = True,
                               informative = False)
    hddmnn_model.sample(n_samples, burn = n_samples/10)
    # 

Supplied model_config specifies params_std_upper for  z as  None.
Changed to 10
 [-----------------100%-----------------] 1000 of 1000 complete in 495.0 sec

In [16]:
md_summ = hddmnn_model.gen_stats().reset_index()

In [17]:
df_summ = hddm.utils.results_long2wide(md_summ, name_col='index', val_col='mean')

KeyError: ignored

In [ ]:
# # Caterpillar Plot: (Parameters recovered ok?)
# hddm.plotting.plot_caterpillar(hddm_model = hddmnn_model,
#                                ground_truth_parameter_dict = full_parameter_dict,
#                                figsize = (8, 8),
#                                columns = 3)
# plt.show()